# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
# your code here
import pandas as pd
ttt_df = pd.read_csv("tic-tac-toe.csv")

display(ttt_df)

print("Looking for emptys")
display(ttt_df.isna().sum())

display(ttt_df.describe())

for column in ttt_df.columns:
    print(column, ttt_df[column].unique())

# I'll not notmalize the data here since I want to split it first

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


Looking for emptys


TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
count,958,958,958,958,958,958,958,958,958,958
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,True
freq,418,378,418,378,458,378,418,378,418,626


TL ['x' 'o' 'b']
TM ['x' 'o' 'b']
TR ['x' 'o' 'b']
ML ['x' 'o' 'b']
MM ['o' 'b' 'x']
MR ['o' 'b' 'x']
BL ['x' 'o' 'b']
BM ['o' 'x' 'b']
BR ['o' 'x' 'b']
class [ True False]


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [26]:
# your code here
from sklearn.model_selection import train_test_split
import tensorflow as tf 

X = ttt_df.copy().drop(["class"], axis=1)
y = ttt_df.copy()["class"]

for column in X.columns:
    X[column] = X[column].apply(lambda x:{'b':0, 'x':1, 'o':2}[x])
# I need to transform labels False == 1 and True == 0
y = y.apply(lambda x: 1 if x == False else 0) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2) # Split data

# Now I normalize the data (after the split)
X_train = tf.keras.utils.normalize(X_train.values, axis=1)
X_test  = tf.keras.utils.normalize(X_test.values, axis=1)
y_train = y_train.values
y_test  = y_test.values

# Prepare the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(9, 1))) 
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model.summary()

model.fit(X_train, y_train, epochs=50, batch_size=16)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 9)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,050 (70.51 KB)

 Trainable params: 18,050 (70.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6162 - loss: 0.6633 
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 867us/step - accuracy: 0.6420 - loss: 0.6244
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - accuracy: 0.6583 - loss: 0.6140
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step - accuracy: 0.6757 - loss: 0.5911
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy: 0.6942 - loss: 0.5569
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - accuracy: 0.7147 - loss: 0.5278
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - accuracy: 0.6994 - loss: 0.5424
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.6933 - loss: 0.5223
Epoch 9/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.7342 - loss: 0.4845
Epoch 10/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - accuracy: 0.7617 - loss: 0.4651
Epoch 11/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 867us/step - accuracy: 0.7579 - loss: 0.4578
Epoch 12/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step

In [24]:
# Evaluate the neural networl
val_loss, val_acc = model.evaluate(X_test, y_test)

print("\nVal Loss:", val_loss)
print("Val Acc:", val_acc)

model.save('tic-tac-toe.keras') # .model is not suported any more, using .keras instead

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8374 - loss: 0.3179  

Val Loss: 0.2925719618797302
Val Acc: 0.859375


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [39]:
# your code here
import tensorflow as tf 
ttt_model = tf.keras.models.load_model("tic-tac-toe.keras")

#predictions = ttt_model.predict([X_test])
import numpy as np
import random

correct_pred = 0
total_pred = 100
for i in range(0, total_pred):
    row = random.randint(0, len(X_test) - 1)
    predictions = ttt_model.predict(np.expand_dims(X_test[row], axis=0))
    print("Row", row, " | Prediction: ", np.argmax(predictions), " | Real value: ", y_test[row])

    if np.argmax(predictions) == y_test[row]:
       correct_pred += 1

print(f"Achieved a { (correct_pred/total_pred) *100 }% of correct prediction on a sample of {total_pred} elements")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Row 76  | Prediction:  1  | Real value:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Row 36  | Prediction:  1  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Row 32  | Prediction:  1  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Row 97  | Prediction:  0  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Row 142  | Prediction:  0  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Row 158  | Prediction:  0  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Row 107  | Prediction:  1  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Row 138  | Prediction:  0  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Row 125  | Prediction:  0  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Row 31  | Prediction:  1  | Real value:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Row 188  | Prediction:  0  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Row 115  | Prediction:  0  | Real value:  0
1/1 ━━━━━━━━━━━━━━━━━

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [67]:
# your code here
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.Input(shape=(9, 1))) 
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))
model2.add(tf.keras.layers.Dense(2, activation="softmax"))
model2.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model2.summary()

model2.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

val_loss, val_acc = model2.evaluate(X_test, y_test)

print("\nVal Loss:", val_loss)
print("Val Acc:", val_acc)

Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_29 (Flatten)            │ (None, 9)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_115 (Dense)               │ (None, 1024)           │        10,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_116 (Dense)               │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_117 (Dense)               │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_118 (Dense)               │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 2)              │         2,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,161,090 (12.06 MB)

 Trainable params: 3,161,090 (12.06 MB)

 Non-trainable params: 0 (0.00 B)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9554 - loss: 0.3257  

Val Loss: 0.25301989912986755
Val Acc: 0.9583333134651184


**Which approach(es) did you find helpful to improve your model performance?**

In [66]:
# your answer here
# Duplicated the amount of layers and incremented the amount of neurons from 128 to 1024